In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [ ]:
# Inicialización
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# Datos
data = []
labels = []

# Captura desde webcam
cap = cv2.VideoCapture(0)

print("Presiona 0 para piedra, 1 para papel, 2 para tijeras. ESC para salir.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(frame_rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            coords = []
            for lm in hand_landmarks.landmark:
                coords.append(lm.x)
                coords.append(lm.y)
            cv2.putText(frame, "Presiona 0/1/2", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            key = cv2.waitKey(1)

            if key in [ord('0'), ord('1'), ord('2')]:
                label = int(chr(key))
                data.append(coords)
                labels.append(label)
                
                # Texto para el gesto
                gesto = ["Piedra", "Papel", "Tijeras"][label]
                
                # Feedback por consola
                print(f"[✓] Gesto guardado: {gesto} ({label})")
                
                # Mostrar el gesto en pantalla por 1 segundo
                cv2.putText(frame, f"Grabado: {gesto}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 3)
                cv2.imshow("Grabando dataset", frame)
                cv2.waitKey(1000)  # Pausa 1 segundo para ver el mensaje


    cv2.imshow("Grabando dataset", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

# Guardar datos
np.save("rps_dataset.npy", np.array(data))
np.save("rps_labels.npy", np.array(labels))
print("Dataset guardado exitosamente.")


Presiona 0 para piedra, 1 para papel, 2 para tijeras. ESC para salir.
[✓] Gesto guardado: Piedra (0)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Tijeras (2)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Piedra (0)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Papel (1)
[✓] Gesto guardado: Piedra (0)
[✓] Gesto guardado: Piedra (0)
[✓] Gesto guardado: Piedra (0)
[✓] Gesto guardado: Piedra (0)
[✓] Gesto guardado: Piedra (0)
[✓] Gesto guardado: Piedra (0)
[✓] Gesto guardado: Piedra (0)
[✓] Gesto guardado: Piedra (0)
[✓] Gesto guardado: Piedra (0)
[✓] Gesto guardado: Piedra (0)
[✓] Gesto guarda

In [ ]:
# 1. Cargar los datos
X = np.load("rps_dataset.npy")  # Shape: (n_samples, 42)
y = np.load("rps_labels.npy")   # Shape: (n_samples,)

# 2. Escalar los datos (opcional pero recomendado)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Codificar etiquetas (one-hot) para clasificación con softmax
y_cat = to_categorical(y, num_classes=3)

# 4. Dividir en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_cat, test_size=0.2, random_state=42)

# 5. Crear la red neuronal
model = Sequential([
    Input(shape=(42,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # 3 clases: piedra, papel, tijeras
])

# 6. Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 7. Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2)

# 8. Evaluar el modelo
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Precisión en test: {acc*100:.2f}%")

# 9. Guardar el modelo entrenado y el scaler
model.save("rps_model.h5")
np.save("rps_scaler_mean.npy", scaler.mean_)
np.save("rps_scaler_scale.npy", scaler.scale_)

print("🧠 Modelo y scaler guardados correctamente.")


Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 189ms/step - accuracy: 0.5970 - loss: 0.9757 - val_accuracy: 0.2500 - val_loss: 1.0875
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6221 - loss: 0.9219 - val_accuracy: 0.2500 - val_loss: 1.0194
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.6889 - loss: 0.8552 - val_accuracy: 0.3750 - val_loss: 0.9663
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.8017 - loss: 0.8421 - val_accuracy: 0.3750 - val_loss: 0.9177
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.8894 - loss: 0.7616 - val_accuracy: 0.5000 - val_loss: 0.8806
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.9102 - loss: 0.7292 - val_accuracy: 0.6250 - val_loss: 0.8419
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.9102 - loss: 0.6778 - val_accuracy: 0.6250 - val_loss: 0.8066
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.9102 - loss: 0.6740 - val_accuracy: 0.6250 - val_loss


🧠 Modelo y scaler guardados correctamente.


In [ ]:
# 1. Cargar modelo y scaler
model = load_model("rps_model.h5")
scaler_mean = np.load("rps_scaler_mean.npy")
scaler_scale = np.load("rps_scaler_scale.npy")

# Función para escalar como en entrenamiento
def scale_input(X):
    return (X - scaler_mean) / scaler_scale

# 2. Inicializar MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# 3. Mapeo de etiquetas
clases = ["Piedra", "Papel", "Tijeras"]

# 4. Captura desde webcam
cap = cv2.VideoCapture(0)

print("🙌 Mostrá un gesto y el sistema lo va a clasificar en tiempo real...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(frame_rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            coords = []
            for lm in hand_landmarks.landmark:
                coords.append(lm.x)
                coords.append(lm.y)

            X_input = scale_input(np.array(coords).reshape(1, -1))
            prediction = model.predict(X_input, verbose=0)
            class_id = np.argmax(prediction)
            prob = prediction[0][class_id]

            label = f"{clases[class_id]} ({prob*100:.1f}%)"
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            cv2.putText(frame, label, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    cv2.imshow("Rock Paper Scissors", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC para salir
        break

cap.release()
cv2.destroyAllWindows()


🙌 Mostrá un gesto y el sistema lo va a clasificar en tiempo real...
